In [106]:
from decimal import Decimal
from pathlib import Path
import rdflib
import tqdm

files = Path('rdf').rglob('*')
files = [f for f in files if f.name[0] != '.' and f.is_file()]
total = rdflib.Graph()
for f in tqdm.tqdm(files):
    data = f.open().read().replace('Property-3A','').replace('Category-3A','').replace('.local', '.org')
    g = rdflib.Graph()
    try:
        g.parse(data=data)
        for t in g:
            try:
                n = Decimal(t[-1].split('/')[-1])
                g.remove(t)
                t = (t[0], t[1], rdflib.Literal(n))
                g.add(t)
            except:
                pass
            if str(t[-1]).strip() == 'Yes' or t[-1].endswith('Yes'):
                g.remove(t)
                t = (t[0], t[1], rdflib.Literal(True))
                g.add(t)
            if str(t[-1]).strip() == 'No' or t[-1].endswith('No'):
                g.remove(t)
                t = (t[0], t[1], rdflib.Literal(True))
                g.add(t)
            for n in t:
                if any(bad in n.lower() for bad in ['swivt', '23aux', '-23', 'ontology', 
                                                    'catalog', 'common_name', 'unknown', 'jpg',
                                                    'article', 'has_search_name',
                                                   ]):
                    g.remove(t)
        total |= g
    except Exception as e:
        print('Failed to parse', f.name)
total.serialize('practicalplants.ttl', format='ttl')

  6%|█████████▊                                                                                                                                                          | 37/618 [00:02<00:41, 14.01it/s]

Failed to parse Echinocystis_lobata


 65%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                         | 400/618 [01:20<01:06,  3.30it/s]

Failed to parse Zanthoxylum_piperitum


 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                 | 492/618 [01:51<00:42,  2.94it/s]

Failed to parse Secale_cereale_ancestrale


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 618/618 [02:45<00:00,  3.75it/s]
